In [1]:
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
classes = ('nodule', 'papule', 'plaque')

In [3]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [4]:
class SkinDiseasesDataset(Dataset):

    def __init__(self, root, transform=None):
        self.transform = transform
        self.root = os.path.join('classes', root)
        self.indexes = self.get_indexes()
        self.classes = {'nodule': 0, 'papule': 1, 'plaque': 2}

    def __len__(self):
        length = 0
        for folder in os.listdir(self.root):
            length += len(os.listdir(os.path.join(self.root, folder)))
        return length

    def get_indexes(self):
        indexes = []
        for filename in (Path(self.root)).rglob('*.*'):
            indexes.append(filename)
        return indexes

    def __getitem__(self, i):
        filename = self.indexes[i]
        label = os.path.dirname(filename).split("\\")[-1]
        label = self.classes[label]
        image = io.imread(filename)
        sample = {'image': image, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [5]:
class ToTensor:

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = image[:, :, :3]  # удаление alpha канала
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image).double(),
                'label': label}

In [6]:
class Resize:

    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = transform.resize(image, (self.output_size, self.output_size))
        return {'image': image, 'label': label}

In [7]:
composed = transforms.Compose(
    [Resize(128), ToTensor()])

In [8]:
dataset = SkinDiseasesDataset(root='train', transform=composed)
trainloader = DataLoader(dataset, batch_size=4,
                         shuffle=True, num_workers=0)
dataset = SkinDiseasesDataset(root='valid', transform=composed)
testloader = DataLoader(dataset, batch_size=4,
                        shuffle=True, num_workers=0)

In [9]:
net = models.resnet18(pretrained=True)

In [10]:
net.fc = nn.Linear(net.fc.in_features, 3)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [12]:
for epoch in range(3):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data['image'], data['label']
        optimizer.zero_grad()
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (i + 1) % 25 == 0:
            print(f'Epoch: {epoch + 1}, {i + 1}%, loss: {running_loss / 25}')
            running_loss = 0.0

print('Finished')

Epoch: 1, 25%, loss: 1.2480095529556274
Epoch: 1, 50%, loss: 1.14595960855484
Epoch: 1, 75%, loss: 1.2333840698003768
Epoch: 2, 25%, loss: 0.8757703787088394
Epoch: 2, 50%, loss: 0.845480443239212
Epoch: 2, 75%, loss: 1.1835740065574647
Epoch: 3, 25%, loss: 0.8578951352834702
Epoch: 3, 50%, loss: 1.0106253147125244
Epoch: 3, 75%, loss: 1.1413308030366898
Finished


In [13]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data['image'], data['label']
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy: {100 * correct / total} %')

Accuracy: 60.0 %
